In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import estimator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_wine
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
wine_data = load_wine()

In [3]:
wine_data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [4]:
print(wine_data['DESCR'])

Wine Data Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- 1) Alcohol
 		- 2) Malic acid
 		- 3) Ash
		- 4) Alcalinity of ash  
 		- 5) Magnesium
		- 6) Total phenols
 		- 7) Flavanoids
 		- 8) Nonflavanoid phenols
 		- 9) Proanthocyanins
		- 10)Color intensity
 		- 11)Hue
 		- 12)OD280/OD315 of diluted wines
 		- 13)Proline
        	- class:
                - class_0
                - class_1
                - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash: 

In [5]:
feat_data = wine_data['data']
labels = wine_data['target']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(feat_data, labels, test_size=0.3, random_state=101)

In [7]:
scaler = MinMaxScaler()

In [8]:
scaler_x_train = scaler.fit_transform(X_train)
scaler_x_test = scaler.transform(X_test)

## Estimator api

In [9]:
feat_cols = [
    tf.feature_column.numeric_column('x', shape=[13])
]

In [10]:
deep_model = estimator.DNNClassifier(
    hidden_units=[13, 13, 13],
    feature_columns=feat_cols,
    n_classes=3,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/0q/36d1fl0d3n1537ld143pvb0r0000gn/T/tmpg67g31zd', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10d824080>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
input_func = estimator.inputs.numpy_input_fn(
    x={'x': scaler_x_train},
    y=y_train,
    shuffle=True,
    batch_size=10,
    num_epochs=10
)

In [12]:
deep_model.train(input_fn=input_func, steps=10_000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/0q/36d1fl0d3n1537ld143pvb0r0000gn/T/tmpg67g31zd/model.ckpt.
INFO:tensorflow:loss = 11.992086, step = 1
INFO:tensorflow:global_step/sec: 695.257
INFO:tensorflow:loss = 0.15139583, step = 101 (0.144 sec)
INFO:tensorflow:Saving checkpoints for 124 into /var/folders/0q/36d1fl0d3n1537ld143pvb0r0000gn/T/tmpg67g31zd/model.ckpt.
INFO:tensorflow:Loss for final step: 0.20069243.


In [13]:
input_func_eval = estimator.inputs.numpy_input_fn(
    x={'x': scaler_x_test},
    y=y_test,
    shuffle=False
)

In [14]:
deep_model.evaluate(input_fn=input_func_eval)

INFO:tensorflow:Starting evaluation at 2018-02-27-14:46:28
INFO:tensorflow:Restoring parameters from /var/folders/0q/36d1fl0d3n1537ld143pvb0r0000gn/T/tmpg67g31zd/model.ckpt-124
INFO:tensorflow:Finished evaluation at 2018-02-27-14:46:28
INFO:tensorflow:Saving dict for global step 124: accuracy = 1.0, average_loss = 0.041080568, global_step = 124, loss = 2.2183506


{'accuracy': 1.0,
 'average_loss': 0.041080568,
 'global_step': 124,
 'loss': 2.2183506}

In [15]:
pred_func_eval = estimator.inputs.numpy_input_fn(
    x={'x': scaler_x_test},
    shuffle=False
)

In [16]:
result = list(deep_model.predict(input_fn=pred_func_eval))

INFO:tensorflow:Restoring parameters from /var/folders/0q/36d1fl0d3n1537ld143pvb0r0000gn/T/tmpg67g31zd/model.ckpt-124


In [17]:
predictions = np.array([pred['class_ids'][0] for pred in result])

In [18]:
np.abs(predictions - y_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [19]:
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        19
          1       1.00      1.00      1.00        22
          2       1.00      1.00      1.00        13

avg / total       1.00      1.00      1.00        54



## Keras

In [20]:
from tensorflow.contrib.keras import models, layers, losses, optimizers, metrics, activations

In [21]:
dnn_keras_model = models.Sequential()

In [22]:
dnn_keras_model.add(layers.Dense(units=13, input_dim=13, activation='relu'))

In [23]:
dnn_keras_model.add(layers.Dense(units=13, activation='relu'))
dnn_keras_model.add(layers.Dense(units=13, activation='relu'))

In [24]:
dnn_keras_model.add(layers.Dense(units=3, activation='softmax'))

In [25]:
dnn_keras_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [26]:
dnn_keras_model.fit(scaler_x_train, y_train, epochs=50)

Epoch 1/50
124/124 [==============================]124/124 [==============================] - 0s 3ms/step - loss: 1.1227 - acc: 0.2097

Epoch 2/50
124/124 [==============================]124/124 [==============================] - 0s 58us/step - loss: 1.1134 - acc: 0.2742

Epoch 3/50
124/124 [==============================]124/124 [==============================] - 0s 58us/step - loss: 1.1055 - acc: 0.3710

Epoch 4/50
124/124 [==============================]124/124 [==============================] - 0s 60us/step - loss: 1.0984 - acc: 0.4435

Epoch 5/50
124/124 [==============================]124/124 [==============================] - 0s 66us/step - loss: 1.0910 - acc: 0.5000

Epoch 6/50
124/124 [==============================]124/124 [==============================] - 0s 84us/step - loss: 1.0841 - acc: 0.5242

Epoch 7/50
124/124 [==============================]124/124 [==============================] - 0s 62us/step - loss: 1.0767 - acc: 0.5161

Epoch 8/50
124/124 [======================

In [27]:
predictions = dnn_keras_model.predict_classes(scaler_x_test)

In [28]:
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       1.00      0.89      0.94        19
          1       0.92      1.00      0.96        22
          2       1.00      1.00      1.00        13

avg / total       0.97      0.96      0.96        54



## Layers API

In [29]:
onehot_y_train = pd.get_dummies(y_train).as_matrix()
onehot_y_test = pd.get_dummies(y_test).as_matrix()

In [30]:
num_feat = 13
num_hidden1 = 13
num_hidden2 = 13
num_outputs = 3
learning_rate = 0.01

In [31]:
from tensorflow.contrib.layers import fully_connected

In [32]:
X = tf.placeholder(tf.float32, shape=[None, num_feat])
y_true = tf.placeholder(tf.int32, shape=[None, 3])

In [33]:
hidden1 = fully_connected(X, num_hidden1, activation_fn=tf.nn.relu)

In [34]:
hidden2 = fully_connected(hidden1, num_hidden2, activation_fn=tf.nn.relu)

In [35]:
output = fully_connected(hidden2, num_outputs)

In [36]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=y_true, logits=output)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [37]:
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [38]:
init = tf.global_variables_initializer()

In [39]:
training_steps = 1000

In [40]:
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(training_steps):
        sess.run(train, feed_dict={
            X: scaler_x_train,
            y_true: onehot_y_train
        })
    
    logits = output.eval(feed_dict={X: scaler_x_test})
    preds = tf.argmax(logits, axis=1)
    results = preds.eval()

In [41]:
print(classification_report(y_test, results))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        19
          1       1.00      1.00      1.00        22
          2       1.00      1.00      1.00        13

avg / total       1.00      1.00      1.00        54

